In [7]:
# 사용방법 - 복사하여 주석만 제거하여 사용
import sys
from pathlib import Path

# 현재 노트북(.ipynb)이 src/ 안에 있으니 상위 폴더(프로젝트 루트)를 추가
BASE_DIR    = Path().resolve().parent.parent    # Notebook이 src/ 안이라면 .parent
config_path = BASE_DIR / 'config' / 'data_paths.yaml'
sys.path.insert(0, str(BASE_DIR))


# 이제 바로 import
from scripts.data_loader import load_data

# 테스트
df_pm10 = load_data('pm10')
# or 
# df = load_data('pm10', section='raw')
df_pm10

,구분,서울특별시,부산광역시,대구광역시,인천광역시,광주광역시,대전광역시,울산광역시,세종특별자치시,수원,...,함양,남해,산청,의령,창녕,합천,도평균.7,제주,서귀포,도평균.8
0,2006.01 월,65,52,60,73,72,56,37,NaN,74,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44,49,NaN
1,2006.02 월,67,55,61,72,65,55,44,NaN,76,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48,45,NaN
2,2006.03 월,81,82,82,83,83,69,81,NaN,95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66,56,NaN
3,2006.04 월,107,93,87,111,82,82,91,NaN,129,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75,64,NaN
4,2006.05 월,63,61,52,69,64,51,55,NaN,71,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49,50,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,2024.06 월,31,26,27,30,23,27,34,28.0,33,...,22.0,23.0,25.0,23.0,26.0,23.0,24.0,22,23,23.0
222,2024.07 월,20,15,16,23,14,15,22,17.0,22,...,15.0,14.0,16.0,18.0,18.0,16.0,16.0,17,21,19.0
223,2024.08 월,24,19,17,27,19,19,23,23.0,26,...,19.0,18.0,19.0,21.0,21.0,19.0,20.0,23,22,23.0
224,2024.09 월,16,15,16,18,16,14,20,17.0,17,...,16.0,13.0,17.0,18.0,18.0,16.0,16.0,21,22,21.0


In [5]:
import pandas as pd 
from datetime import datetime

df = load_data('pm10', section='raw')
df_long = df.melt(id_vars='구분', var_name='region', value_name='pm10')
df_long['pm10'] = pd.to_numeric(df_long['pm10'], errors='coerce')
missing_counts = df_long.groupby('region')['pm10'].apply(lambda x: x.isna().sum())
valid_regions = missing_counts[missing_counts < 10].index
df_filtered = df_long[df_long['region'].isin(valid_regions)]

avg_pm10_filtered = (
    df_filtered.groupby('region')['pm10']
    .mean()
    .sort_values(ascending=True)
    .head(30)
    .reset_index()
)

avg_pm10_filtered.columns = ['지역', '평균 PM10 농도']
avg_pm10_filtered.insert(0, '순위', range(1, len(avg_pm10_filtered) + 1))

display("결측치 10개 미만 지역 상위 30 미세먼지 평균 농도 도시", avg_pm10_filtered)

today = datetime.now().strftime('%Y%m%d')
filename = f"pm10_top30_{today}_v1.csv"

out_dir  = BASE_DIR / 'data' / 'processed'
out_path = out_dir / filename

out_dir.mkdir(parents=True, exist_ok=True)
# avg_pm10_filtered.to_csv(out_path, index=False, encoding='utf-8-sig') # 실제 저장 코드 

print(f"Top30 PM10 결과를 저장했습니다: {out_path}")

'결측치 10개 미만 지역 상위 30 미세먼지 평균 농도 도시'

,순위,지역,평균 PM10 농도
0,1,순천,33.848889
1,2,여수,35.261062
2,3,서귀포,35.809735
3,4,동해,37.208889
4,5,광양,37.336283
5,6,경주,39.644444
6,7,김해,39.827434
7,8,목포,39.831858
8,9,제주,39.938053
9,10,양산,40.247788


Top30 PM10 결과를 저장했습니다: C:\team_project\dust\data\processed\pm10_top30_20250529_v1.csv


In [6]:
# 컬럼명 수정

df_region = load_data('region_map', section='reference')
df_avgcode = load_data('avgcode_map', section='reference')

df_region = df_region.rename(columns={
    '시군구코드': 'region_id',
    '시도명': 'province',
    '시군구명': 'region_name'
})

# 다시 매핑 기준 설정 및 매핑 수행
region_names = df_region['region_name'].unique().tolist()
avgcode_names = df_avgcode['avg_code'].unique().tolist()
pm10_columns = df_pm10.columns.tolist()

column_mapping = {}
for col in pm10_columns:
    col_stripped = col.strip()
    if col_stripped in region_names:
        column_mapping[col] = col_stripped
    elif col_stripped in avgcode_names:
        column_mapping[col] = col_stripped
    else:
        matched = [r for r in region_names if col_stripped in r]
        if len(matched) == 1:
            column_mapping[col] = matched[0]
        else:
            column_mapping[col] = col

# 컬럼명 변경 및 저장
df_pm10_renamed = df_pm10.rename(columns=column_mapping)
today = datetime.now().strftime('%Y%m%d')
filename = f"pm10_rename_region_{today}_v1.csv"

out_dir  = BASE_DIR / 'data' / 'processed'
out_path = out_dir / filename

out_dir.mkdir(parents=True, exist_ok=True)

df_pm10_renamed.to_excel(out_path, index=False)

In [10]:
# 시도명(province)도 포함하여 체크하도록 확장
region_names = df_region['region_name'].str.strip().unique().tolist()
province_names = df_region['province'].str.strip().unique().tolist()

# pm10의 지역 컬럼에서 구분 제외하고 정리
pm10_columns = df_pm10.columns.tolist()
pm10_areas = [col.strip() for col in pm10_columns if col != '구분']

# 매칭 여부 확인
matched = []
unmatched = []

for area in pm10_areas:
    if area in region_names or area in province_names:
        matched.append(area)
    else:
        unmatched.append(area)

# 매칭 결과 요약
df_check = pd.DataFrame({
    '지역명': pm10_areas,
    'region_name_매칭': [area in region_names for area in pm10_areas],
    'province_매칭': [area in province_names for area in pm10_areas],
    '최종_매칭여부': [area in region_names or area in province_names for area in pm10_areas]
})

print(df_check)

       지역명  region_name_매칭  province_매칭  최종_매칭여부
0    서울특별시           False         True     True
1    부산광역시           False         True     True
2    대구광역시           False         True     True
3    인천광역시           False         True     True
4    광주광역시           False         True     True
..     ...             ...          ...      ...
169     합천           False        False    False
170  도평균.7           False        False    False
171     제주           False        False    False
172    서귀포           False        False    False
173  도평균.8           False        False    False

[174 rows x 4 columns]


In [18]:
from collections import defaultdict

# 1. PM10 지역 컬럼 정리
pm10_area_cols = [col for col in df_pm10.columns if col != '구분']
df_region = df_region.rename(columns={
    '시도명': 'province',
    '시군구명': 'region_name',
    '지역코드': 'region_id'
})
region_ref = df_region['region_name'].unique().tolist()

mapped_cols = {}
unmatched_cols = []
region_group_counter = defaultdict(list)

# 2. 지역명 직접 매핑 (예: 수원 → 수원시)
for col in pm10_area_cols:
    matched = [r for r in region_ref if r in col or col in r]
    if matched:
        base = sorted(matched, key=len)[0]  # 가장 짧은 시 단위
        mapped_cols[col] = base
        region_group_counter[base].append(col)
    else:
        unmatched_cols.append(col)

# 3. 시도명/도평균 매핑
province_map = df_region[['province', 'region_id']].drop_duplicates()
avgcode_map = df_avgcode.rename(columns={'avg_code': 'area'})
avgcode_map = avgcode_map.merge(province_map, left_on='province_name', right_on='province', how='left')

province_dict = dict(zip(province_map['province'], province_map['region_id']))
avgcode_dict = dict(zip(avgcode_map['area'], avgcode_map['region_id']))

mapped_cols_full = mapped_cols.copy()

for col in unmatched_cols:
    if col in province_dict:
        mapped_cols_full[col] = province_dict[col]
    elif col in avgcode_dict:
        mapped_cols_full[col] = avgcode_dict[col]
    else:
        mapped_cols_full[col] = col  # 여전히 미매핑 상태

# 4. 컬럼명 변경 (순서 유지)
df_pm10_final = df_pm10.copy()
df_pm10_final.columns = [mapped_cols_full.get(col, col) for col in df_pm10.columns]

# 5. 중복 컬럼 평균 처리
# 1. '구분' 제외한 수치형 데이터프레임
value_df = df_pm10_final.drop(columns='구분')

# 2. 숫자형으로 변환 (에러 방지용)
value_df = value_df.apply(pd.to_numeric, errors='coerce')

# 3. 같은 컬럼명이 있는 경우 평균 내기
# key: 컬럼명, value: 해당 컬럼들의 평균
grouped_avg = value_df.groupby(by=value_df.columns, axis=1).mean()

# 4. 다시 '구분'과 병합
df_result = pd.concat([df_pm10[['구분']], grouped_avg], axis=1)

print(df_result)

df_result = df_result.rename(columns=column_mapping)
today = datetime.now().strftime('%Y%m%d')
filename = f"pm10_rename_region_{today}_v1.csv"

out_dir  = BASE_DIR / 'data' / 'processed'
out_path = out_dir / filename

out_dir.mkdir(parents=True, exist_ok=True)

df_result.to_excel(out_path, index=False)




C:\Users\김남준\AppData\Local\Temp\ipykernel_27652\3721906446.py:57: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  grouped_avg = value_df.groupby(by=value_df.columns, axis=1).mean()


            구분  11740  26710  27720  28720  29200  30230  31710  36110  41830  \
0    2006.01 월   65.0   52.0   60.0   73.0   72.0   56.0   37.0    NaN    NaN   
1    2006.02 월   67.0   55.0   61.0   72.0   65.0   55.0   44.0    NaN    NaN   
2    2006.03 월   81.0   82.0   82.0   83.0   83.0   69.0   81.0    NaN    NaN   
3    2006.04 월  107.0   93.0   87.0  111.0   82.0   82.0   91.0    NaN    NaN   
4    2006.05 월   63.0   61.0   52.0   69.0   64.0   51.0   55.0    NaN    NaN   
..         ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
221  2024.06 월   31.0   26.0   27.0   30.0   23.0   27.0   34.0   28.0   32.0   
222  2024.07 월   20.0   15.0   16.0   23.0   14.0   15.0   22.0   17.0   21.0   
223  2024.08 월   24.0   19.0   17.0   27.0   19.0   19.0   23.0   23.0   25.0   
224  2024.09 월   16.0   15.0   16.0   18.0   16.0   14.0   20.0   17.0   18.0   
225  2024.10 월   23.0   17.0   20.0   28.0   22.0   21.0   20.0   23.0   25.0   

     ...   함양군   함평군   합천군 

In [28]:
# pm10의 전처리 컬럼명 mapping 작업 
df_region = load_data('region_map', section='reference')
df_avgcode = load_data('avgcode_map', section='reference')
df_pm10 = load_data('pm10', section='raw')

original_columns = df_pm10.columns.tolist()
area_cols = [col for col in original_columns if col != '구분']

# 결과 저장용
col_mapping = {}
unmatched_cols = []

for col in area_cols:
    new_col = None

    # 1) 시도명으로 바로 존재
    if col in province_names:
        new_col = col

    # 2) 도평균 계열
    elif col in avgcode_map:
        new_col = avgcode_map[col]

    # 3) 시군구 앞글자 2개 매핑
    else:
        prefix = col[:2]
        matched = [name for name in region_names if name.startswith(prefix)]
        if matched:
            # 가장 짧은 이름 (상위 행정구역 우선)
            new_col = sorted(matched, key=len)[0]

    # 4) 마지막 fallback
    if new_col:
        col_mapping[col] = new_col
    else:
        unmatched_cols.append(col)

# 5) 하드코딩 추가
manual_map = {
    '청원': '청원구',
    '마산': '마산합포구',
    '진해': '진해구'
}
col_mapping.update(manual_map)

# 매핑 적용
df_pm10_renamed = df_pm10.rename(columns=col_mapping)

# 변경된 컬럼만 확인용
mapped_cols = list(col_mapping.keys()) + list(manual_map.keys())
updated_columns = [col for col in df_pm10_renamed.columns if col != '구분']


print(mapped_cols)

df_result = df_pm10_renamed.rename(columns=column_mapping)
today = datetime.now().strftime('%Y%m%d')
filename = f"pm10_rename_region_{today}.xlsx"

out_dir  = BASE_DIR / 'data' / 'processed'
out_path = out_dir / filename

out_dir.mkdir(parents=True, exist_ok=True)
df_result.to_excel(out_path, index=False)


['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시', '세종특별자치시', '수원', '안양', '성남', '의정부', '광명', '안산', '평택', '과천', '구리', '의왕', '고양', '광주', '군포', '시흥', '부천', '남양주', '용인', '김포', '오산', '하남', '화성', '양주', '동두천', '안성', '이천', '파주', '포천', '여주', '연천', '가평', '양평', '도평균', '양구', '고성', '정선', '횡성', '춘천', '강릉', '원주', '동해', '삼척', '평창', '양양', '속초', '영월', '철원', '홍천', '화천', '태백', '인제', '도평균.1', '괴산', '음성', '청주', '충주', '제천', '단양', '진천', '옥천', '영동', '증평', '보은', '도평균.2', '천안', '당진', '서산', '아산', '논산', '태안', '보령', '홍성', '공주', '부여', '청양', '금산', '예산', '계룡', '서천', '도평균.3', '임실', '전주', '군산', '익산', '남원', '정읍', '고창', '부안', '김제', '완주', '진안', '무주', '순창', '장수', '도평균.4', '화순', '여수', '광양', '순천', '목포', '영암', '나주', '담양', '장성', '해남', '영광', '장흥', '진도', '완도', '함평', '고흥', '신안', '무안', '강진', '곡성', '구례', '보성', '도평균.5', '영덕', '영천', '의성', '울릉도', '포항', '구미', '김천', '경주', '안동', '영주', '경산', '상주', '칠곡', '울진', '봉화', '고령', '군위', '성주', '예천', '청도', '청송', '영양', '문경', '도평균.6', '창원', '진주', '하동', '김해', '양산', '거제', '사천', '밀양', '통영', '고

In [32]:
# pm25의 전처리 컬럼명 mapping 작업 
df_region = load_data('region_map', section='reference')
df_avgcode = load_data('avgcode_map', section='reference')
df_pm25 = load_data('pm25', section='raw')

original_columns = df_pm25.columns.tolist()
area_cols = [col for col in original_columns if col != '구분']

# 결과 저장용
col_mapping = {}
unmatched_cols = []

for col in area_cols:
    new_col = None

    # 1) 시도명으로 바로 존재
    if col in province_names:
        new_col = col

    # 2) 도평균 계열
    elif col in avgcode_map:
        new_col = avgcode_map[col]

    # 3) 시군구 앞글자 2개 매핑
    else:
        prefix = col[:2]
        matched = [name for name in region_names if name.startswith(prefix)]
        if matched:
            # 가장 짧은 이름 (상위 행정구역 우선)
            new_col = sorted(matched, key=len)[0]

    # 4) 마지막 fallback
    if new_col:
        col_mapping[col] = new_col
    else:
        unmatched_cols.append(col)

# 5) 하드코딩 추가
manual_map = {
    '청원': '청원구',
    '마산': '마산합포구',
    '진해': '진해구'
}
col_mapping.update(manual_map)

# 매핑 적용
df_pm25_renamed = df_pm25.rename(columns=col_mapping)

# 변경된 컬럼만 확인용
mapped_cols = list(col_mapping.keys()) + list(manual_map.keys())
updated_columns = [col for col in df_pm25_renamed.columns if col != '구분']


print(mapped_cols)

df_result = df_pm25_renamed.rename(columns=column_mapping)
df_result.isnull().sum()
# today = datetime.now().strftime('%Y%m%d')
# filename = f"pm25_rename_region_{today}.xlsx"

# out_dir  = BASE_DIR / 'data' / 'processed'
# out_path = out_dir / filename

# out_dir.mkdir(parents=True, exist_ok=True)

# df_result.to_excel(out_path, index=False)




['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시', '세종특별자치시', '수원', '안양', '성남', '의정부', '광명', '안산', '평택', '과천', '구리', '의왕', '고양', '광주', '군포', '시흥', '부천', '남양주', '용인', '김포', '오산', '하남', '화성', '양주', '동두천', '안성', '이천', '파주', '포천', '여주', '연천', '가평', '양평', '도평균', '양구', '고성', '정선', '횡성', '춘천', '강릉', '원주', '동해', '삼척', '평창', '양양', '속초', '영월', '철원', '홍천', '화천', '태백', '인제', '도평균.1', '괴산', '음성', '청주', '충주', '제천', '단양', '진천', '옥천', '영동', '증평', '보은', '도평균.2', '천안', '당진', '서산', '아산', '논산', '태안', '보령', '홍성', '공주', '부여', '청양', '금산', '예산', '계룡', '서천', '도평균.3', '임실', '전주', '군산', '익산', '남원', '정읍', '고창', '부안', '김제', '완주', '진안', '무주', '순창', '장수', '도평균.4', '화순', '여수', '광양', '순천', '목포', '영암', '나주', '담양', '장성', '해남', '영광', '장흥', '진도', '완도', '함평', '고흥', '신안', '무안', '강진', '곡성', '구례', '보성', '도평균.5', '영덕', '영천', '의성', '울릉도', '포항', '구미', '김천', '경주', '안동', '영주', '경산', '상주', '칠곡', '울진', '봉화', '고령', '군위', '성주', '예천', '청도', '청송', '영양', '문경', '도평균.6', '창원', '진주', '하동', '김해', '양산', '거제', '사천', '밀양', '통영', '고

구분          0
서울특별시       0
부산광역시       0
대구광역시       0
인천광역시       0
           ..
합천군        59
경상남도        0
제주시         0
서귀포시       10
제주특별자치도    10
Length: 172, dtype: int64

In [31]:
import pandas as pd


today = datetime.now().strftime('%Y%m%d')
filename = f"reference_region_mapping_sum_{today}.xlsx"

FILENAME = filename
# 파일 불러오기
region_df = load_data('region_map', section='reference')

# 1. 시도명 코드 매핑용 번호 생성 (예: 서울특별시 = 10000, 부산광역시 = 20000 ...)
unique_provinces = region_df["시도명"].dropna().unique()
province_codes = {prov: (i + 1) * 10000 for i, prov in enumerate(sorted(unique_provinces))}

# 2. 시도코드 컬럼 추가
region_df["시도코드"] = region_df["시도명"].map(province_codes)

# 3. 시군구명(통합) 컬럼 추가 (구, 군 구분 없는 상위 시도로 통합)
def make_integrated_name(row):
    if "구" in str(row["시군구명"]) or "군" in str(row["시군구명"]):
        return row["시도명"]
    else:
        return row["시군구명"]

region_df["시군구명_통합"] = region_df.apply(make_integrated_name, axis=1)

df_result = region_df.rename(columns=column_mapping)

# 파일저장 
out_dir  = BASE_DIR / 'data' / 'processed'
out_path = out_dir / filename
out_dir.mkdir(parents=True, exist_ok=True)
df_result.to_excel(out_path, index=False)